<a href="https://colab.research.google.com/github/RitwizSarma/ocr-for-policy/blob/main/AmexDecisionSci_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preface

## Libraries

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
tqdm.pandas()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier as gbc
from sklearn.ensemble import GradientBoostingRegressor as gbr
from xgboost import XGBClassifier as xgbc
from xgboost import XGBRegressor as xgbr
from lightgbm import LGBMClassifier as lgbmc
from lightgbm import LGBMRegressor as lgbmr
!pip install catboost
from catboost import CatBoostClassifier as cbc
from catboost import CatBoostRegressor as cbr

from sklearn.model_selection import train_test_split as TTS
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.metrics import classification_report as classrep

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.1 MB/s eta 0:00:00


<h3> Some notes </h3>

Datasets given:
 - Batsman level \[for use in feature engg\]
 - Bowler level \[for use in feature engg\]
 - Match level \[for use in feature engg\]
 - Train dataset \[needed for running models\]
 - Test dataset \[needed for running models\]

# Exploratory data analysis

## Reading data

In [ ]:
bt_df = pd.read_csv("/content/663e2b548c98c_batsman_level_scorecard.csv")
bwl_df = pd.read_csv("/content/663e2b2c60743_bowler_level_scorecard.csv")
match_df = pd.read_csv("/content/664389efa0868_match_level_scorecard.csv")
train = pd.read_csv("/content/663e2b6d54457_train_data_with_samplefeatures.csv")
test = pd.read_csv("/content/6644a1e287df6_test_data_with_samplefeatures.csv")

## Batsman-level dataset

FileNotFoundError: [Errno 2] No such file or directory: '/content/663e2b548c98c_batsman_level_scorecard.csv'

In [ ]:
bt_df

,match id,batsman,batsman_id,batsman_details,is_batsman_captain,is_batsman_keeper,inning,runs,balls_faced,over_faced_first,...,out_by_bowler,out_by_fielder,bowler_id,bowler_details,is_bowler_keeper,is_bowler_captain,strike_rate,Fours,Sixes,match_dt
0,8638034,KD Ce,7907451.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,7,5,1.1,...,JS Nm,TM Jn,2486896.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,140.00,1.0,NaN,2021-01-01
1,8638034,TL St,4381761.0,NZ:Right-hand bat:None:,0.0,1.0,1,46,46,1.2,...,R Ra,MG Bl,6718382.0,NZ:Left-hand bat:Slow left-arm orthodox:,0.0,0.0,100.00,4.0,1.0,2021-01-01
2,8638034,HR Cr,4949790.0,NZ:Right-hand bat:Right-arm offbreak:,0.0,0.0,1,9,10,12.1,...,PF Yd,R Ra,4950294.0,NZ:Right-hand bat:Legbreak googly:,0.0,0.0,90.00,NaN,NaN,2021-01-01
3,8638034,BR Hn,3834305.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,28,22,13.6,...,HK Bt,FH An,1585464.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,127.27,3.0,NaN,2021-01-01
4,8638034,SC Kn,3776849.0,NZ:Right-hand bat:Right-arm fast-medium:,0.0,0.0,1,18,13,17.2,...,NaN,NaN,NaN,NaN,NaN,NaN,138.46,NaN,1.0,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24478,9433633,CP Wd,2173688.0,ENG:Right-hand bat:Left-arm medium-fast:,0.0,0.0,1,1,1,20.4,...,NaN,NaN,NaN,NaN,NaN,NaN,100.00,NaN,NaN,2023-06-20
24479,9433633,JJ Wy,5241564.0,ENG:Right-hand bat:Right-arm offbreak:,0.0,0.0,1,37,28,6.6,...,TJ Sh,G Rn,1707628.0,ENG:Right-hand bat:Slow left-arm orthodox:,0.0,0.0,132.14,4.0,1.0,2023-06-20
24480,9433633,G Rn,4898074.0,SA:Right-hand bat:None:,0.0,1.0,2,1,6,1.1,...,JA Tr,LA Dn,8833075.0,ENG:Right-hand bat:Right-arm fast-medium:,0.0,0.0,16.67,NaN,NaN,2023-06-20
24481,9433633,BJ Ws,8048074.0,ENG:Right-hand bat:None:,0.0,0.0,2,43,22,2.3,...,NaN,NaN,NaN,NaN,NaN,NaN,195.45,3.0,3.0,2023-06-20


In [ ]:
bt_df.columns

Index(['match id', 'batsman', 'batsman_id', 'batsman_details',
       'is_batsman_captain', 'is_batsman_keeper', 'inning', 'runs',
       'balls_faced', 'over_faced_first', 'wicket kind', 'out_by_bowler',
       'out_by_fielder', 'bowler_id', 'bowler_details', 'is_bowler_keeper',
       'is_bowler_captain', 'strike_rate', 'Fours', 'Sixes', 'match_dt'],
      dtype='object')

In [ ]:
train_bt = bt_df[bt_df['match id'].isin(train['match id'])]

In [ ]:
train_bt[train_bt['wicket kind'].isna()]

,match id,batsman,batsman_id,batsman_details,is_batsman_captain,is_batsman_keeper,inning,runs,balls_faced,over_faced_first,...,out_by_bowler,out_by_fielder,bowler_id,bowler_details,is_bowler_keeper,is_bowler_captain,strike_rate,Fours,Sixes,match_dt
10031,9331181,VR An,2526390.0,IND:Right-hand bat:Right-arm fast:,0.0,0.0,1,9,6,18.6,...,NaN,NaN,NaN,NaN,NaN,NaN,150.00,NaN,1.0,2022-10-20
10034,9331181,P Ca,3500958.0,IND:Right-hand bat:Legbreak googly:,0.0,0.0,2,61,48,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,127.08,4.0,2.0,2022-10-20
10035,9331181,R Dn,2035102.0,IND:Right-hand bat:Right-arm medium-fast:,1.0,0.0,2,4,7,15.4,...,NaN,NaN,NaN,NaN,NaN,NaN,57.14,NaN,NaN,2022-10-20
10042,8797060,AU Rd,1711492.0,ENG:Right-hand bat:Legbreak:,0.0,0.0,1,2,3,19.6,...,NaN,NaN,NaN,NaN,NaN,NaN,66.67,NaN,NaN,2022-01-23
10043,8797060,S Md,4507208.0,ENG:Right-hand bat:Right-arm fast-medium:,0.0,0.0,1,7,4,20.3,...,NaN,NaN,NaN,NaN,NaN,NaN,175.00,1.0,NaN,2022-01-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24469,9516695,M Jn,4874820.0,SA:Right-hand bat:Left-arm medium-fast:,0.0,0.0,2,3,2,19.6,...,NaN,NaN,NaN,NaN,NaN,NaN,150.00,NaN,NaN,2023-05-07
24473,9433633,RA Wy,2020332.0,ENG:Left-hand bat:Left-arm medium:,0.0,0.0,1,41,25,12.5,...,NaN,NaN,NaN,NaN,NaN,NaN,164.00,3.0,2.0,2023-06-20
24478,9433633,CP Wd,2173688.0,ENG:Right-hand bat:Left-arm medium-fast:,0.0,0.0,1,1,1,20.4,...,NaN,NaN,NaN,NaN,NaN,NaN,100.00,NaN,NaN,2023-06-20
24481,9433633,BJ Ws,8048074.0,ENG:Right-hand bat:None:,0.0,0.0,2,43,22,2.3,...,NaN,NaN,NaN,NaN,NaN,NaN,195.45,3.0,3.0,2023-06-20


In [ ]:
train_bt['wicket kind'].unique()

array(['run out', 'caught', nan, 'bowled', 'caught and bowled', 'lbw',
       'stumped', 'hit wicket', 'retired hurt', 'retired out',
       'retired not out'], dtype=object)

In [ ]:
keepers = train_bt[train_bt['wicket kind']=='stumped']['out_by_fielder'].unique()

In [ ]:
train_bt[train_bt['wicket kind']=='retired out']

,match id,batsman,batsman_id,batsman_details,is_batsman_captain,is_batsman_keeper,inning,runs,balls_faced,over_faced_first,...,out_by_bowler,out_by_fielder,bowler_id,bowler_details,is_bowler_keeper,is_bowler_captain,strike_rate,Fours,Sixes,match_dt
14170,9684800,N Jn,7341704.0,IND:Right-hand bat:None:,0.0,1.0,1,15,13,16.3,...,Ah Kn,NaN,4859490.0,IND:Right-hand bat:Right-arm fast-medium:,0.0,0.0,115.38,NaN,1.0,2023-10-25
14826,9084886,CR Be,3200756.0,WI:Right-hand bat:Right-arm fast-medium:,1.0,0.0,1,17,11,5.6,...,JT Bl,NaN,2904943.0,ENG:Right-hand bat:Right-arm fast-medium:,0.0,0.0,154.55,2.0,1.0,2022-06-05
14833,9084886,SR Pl,130437.0,ENG:Right-hand bat:Slow left-arm orthodox:,0.0,0.0,2,2,2,8.2,...,CN Ms,NaN,2855138.0,ENG:Right-hand bat:Right-arm medium:,0.0,0.0,100.00,NaN,NaN,2022-06-05
16628,9587038,SE Rd,6401800.0,WI:Left-hand bat:Right-arm fast-medium:,1.0,0.0,1,65,27,14.3,...,JO Hr,NaN,2740408.0,WI:Right-hand bat:Right-arm medium-fast:,0.0,0.0,240.74,5.0,5.0,2023-09-03
18393,9128475,R An,184960.0,IND:Right-hand bat:Right-arm offbreak:,0.0,0.0,1,28,23,10.6,...,Ah Kn,NaN,4859490.0,IND:Right-hand bat:Right-arm fast-medium:,0.0,0.0,121.74,NaN,2.0,2022-04-10
21878,9271681,KA Ba,6710850.0,SL:Right-hand bat:Legbreak:,0.0,0.0,2,44,39,6.6,...,Zn Kn,NaN,8907338.0,PAK:Right-hand bat:Right-arm fast:,0.0,0.0,112.82,3.0,NaN,2022-12-10


In [ ]:
train_bt.batsman_id.nunique()

1757

In [ ]:
train_bt.isna().sum()

match id                 0
batsman                  0
batsman_id               0
batsman_details          0
is_batsman_captain       0
is_batsman_keeper        0
inning                   0
runs                     0
balls_faced              0
over_faced_first         0
wicket kind           3139
out_by_bowler         3139
out_by_fielder        6256
bowler_id             3596
bowler_details        3596
is_bowler_keeper      3596
is_bowler_captain     3596
strike_rate              8
Fours                 6118
Sixes                 9079
match_dt                 0
dtype: int64

In [ ]:
train_bt.groupby(by='match id').runs.sum()

match id
8797053    199
8797060    321
8797067    403
8797074    332
8844324    232
          ... 
9866338    367
9866345    323
9866366    337
9866373     92
9887863    212
Name: runs, Length: 948, dtype: int64

## Bowlers-level dataset

In [ ]:
bwl_df

,match id,bowler,bowler_id,bowler_details,is_bowler_captain,is_bowler_keeper,inning,runs,wicket_count,balls_bowled,economy,maiden,dots,Fours,Sixes,wides,noballs,match_dt
0,8638034,BV Ss,6718396.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,21,0,12,10.50,0,3.0,1,1,3,1,2021-01-01
1,8638034,HK Bt,1585464.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,1,34,1,24,8.50,0,7.0,3,1,2,0,2021-01-01
2,8638034,JS Nm,2486896.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,1,23,2,24,5.75,0,12.0,1,1,0,0,2021-01-01
3,8638034,LV vn Bk,3083667.0,NED:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,31,1,24,7.75,0,9.0,4,0,2,0,2021-01-01
4,8638034,PF Yd,4950294.0,NZ:Right-hand bat:Legbreak googly:,0.0,0.0,1,21,1,24,5.25,0,8.0,0,0,1,0,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18534,9433633,BC Hl,1482249.0,ENG:Right-hand bat:Right-arm medium:,0.0,0.0,2,16,0,6,16.00,0,1.0,0,2,0,0,2023-06-20
18535,9433633,CP Wd,2173688.0,ENG:Right-hand bat:Left-arm medium-fast:,0.0,0.0,2,7,0,12,3.50,0,6.0,0,0,0,0,2023-06-20
18536,9433633,JA Tr,8833075.0,ENG:Right-hand bat:Right-arm fast-medium:,0.0,0.0,2,17,1,12,8.50,0,9.0,1,2,1,0,2023-06-20
18537,9433633,LA Dn,1482998.0,ENG:Right-hand bat:Slow left-arm orthodox:,0.0,0.0,2,29,1,12,14.50,0,4.0,4,1,0,0,2023-06-20


In [ ]:
bwl_df['bowler_details'].unique()

array(['NZ:Right-hand bat:Right-arm medium-fast:',
       'NZ:Left-hand bat:Right-arm medium-fast:',
       'NED:Right-hand bat:Right-arm medium-fast:',
       'NZ:Right-hand bat:Legbreak googly:',
       'NZ:Left-hand bat:Slow left-arm orthodox:',
       'NZ:Right-hand bat:Right-arm medium:',
       'NZ:Right-hand bat:Right-arm offbreak:',
       'NZ:Right-hand bat:Right-arm fast-medium:',
       'AUS:Right-hand bat:Right-arm fast-medium:',
       'AUS:Right-hand bat:Right-arm offbreak:',
       'AUS:Right-hand bat:Right-arm medium-fast:',
       'AUS:Right-hand bat:Left-arm fast-medium:',
       'AUS:Right-hand bat:Legbreak:',
       'PAK:Left-hand bat:Slow left-arm orthodox:',
       'AUS:Left-hand bat:Right-arm fast-medium:',
       'AFG:Right-hand bat:Right-arm offbreak:',
       'AUS:Right-hand bat:Legbreak googly:',
       'ENG:Right-hand bat:Right-arm medium-fast:',
       'AUS:Left-hand bat:Slow left-arm orthodox:',
       'ITA:Right-hand bat:Right-arm offbreak:',
       'AUS:

In [ ]:
bwl_df.columns

Index(['match id', 'bowler', 'bowler_id', 'bowler_details',
       'is_bowler_captain', 'is_bowler_keeper', 'inning', 'runs',
       'wicket_count', 'balls_bowled', 'economy', 'maiden', 'dots', 'Fours',
       'Sixes', 'wides', 'noballs', 'match_dt'],
      dtype='object')

In [ ]:
bwl_df.groupby(by='bowler_id').is_bowler_captain.sum()

bowler_id
34061.0       4.0
37351.0       1.0
41740.0       5.0
49496.0       0.0
55299.0      18.0
             ... 
9822392.0     0.0
9822413.0     0.0
9822420.0     0.0
9822441.0     0.0
9888584.0     0.0
Name: is_bowler_captain, Length: 1541, dtype: float64

In [ ]:
bwl_df.groupby(by='bowler_id').wicket_count.sum().sort(ascending=False)

In [ ]:
train_bwl = bwl_df[bwl_df['match id'].isin(train['match id'])]

In [ ]:
train_bwl['match id'].nunique()

948

In [ ]:
train_bwl.isna().sum()

match id             0
bowler               0
bowler_id            0
bowler_details       0
is_bowler_captain    0
is_bowler_keeper     0
inning               0
runs                 0
wicket_count         0
balls_bowled         0
economy              0
maiden               0
dots                 0
Fours                0
Sixes                0
wides                0
noballs              0
match_dt             0
dtype: int64

## Match-level dataset

In [ ]:
match_df.shape

In [ ]:
match_df.columns

Index(['match id', 'team1', 'team2', 'winner', 'by', 'win amount',
       'toss winner', 'toss decision', 'venue', 'city', 'match_dt', 'lighting',
       'series_name', 'season', 'ground_id', 'umpire1', 'umpire2',
       'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_runs',
       'inning2_wickets', 'inning2_balls', 'team1_id', 'team1_roster_ids',
       'team2_id', 'team2_roster_ids', 'series_type', 'winner_id',
       'player_of_the_match_id'],
      dtype='object')

In [ ]:
match_df['win amount'].unique()

array([  9.,   7.,   4.,   2.,  21.,  10.,  96.,   6.,   5.,  60.,  86.,
        17.,  39.,  53.,  80.,  18.,  32.,  11.,  43.,  66.,  29.,   3.,
        76.,   8.,  73., 112., 130.,  79., 117.,  51.,  26., 111.,  89.,
       100.,  45.,  90.,  54.,  13.,  37.,  77., 131.,  12.,  22., 110.,
        15.,  59.,  82.,  46.,  25., 103.,  49.,  30.,  36.,  27.,  14.,
        16.,  24.,  41.,  64.,  50.,  48.,  47.,  28.,  65.,  38.,  19.,
        69.,   1.,  34.,  55.,  23.,  20.,  61.,  40.,  94.,  35.,  67.,
        31.,  42.,  70.,  63.,  33., 114.,  75.,  56.,  74.,  78.,  93.,
       120.,  52.,  62.,  44.,  81.,  72., 167.,  92., 152., 102.,  58.,
        85.,  95., 101., 125., 144.,  98., 106.,  68., 124., 105., 128.,
       132., 151.,  71., 122., 118.,  84., 168., 119.,  83., 109., 104.,
       133.,  87.,  88.,  57., 129.,  91.])

In [ ]:
match_df.groupby(by='venue').sum

venue
 Ed Hs Il Sl Gd                                                         0
AA Si                                                                   0
AA Sm Ba                                                                0
Aa Sm Ln                                                       Launceston
Ac 2  Ct Gd                                                             0
                                              ...                        
Ws Ct Gd Wk              WindhoekWindhoekWindhoekWindhoekWindhoekWindhoek
Wt Ed Pk Il Ct Sm Da                                         DohaDohaDoha
Zd Ct Sm Au Di          Abu DhabiAbu DhabiAbu DhabiAbu DhabiAbu DhabiA...
Zg Uy of Ty Ct Fd                        HangzhouHangzhouHangzhouHangzhou
Zr Ad Cy Sm Cm          ChattogramChattogramChattogramChattogramChatto...
Name: city, Length: 181, dtype: object

In [ ]:
match_df.isna().sum()

match id                    0
team1                       0
team2                       0
winner                      0
by                          0
win amount                  0
toss winner                 0
toss decision               0
venue                       0
city                      178
match_dt                    0
lighting                    0
series_name                 0
season                      0
ground_id                   0
umpire1                   181
umpire2                   181
inning1_runs                0
inning1_wickets             0
inning1_balls               0
inning2_runs                0
inning2_wickets             0
inning2_balls               0
team1_id                    0
team1_roster_ids            0
team2_id                    0
team2_roster_ids            0
series_type                 0
winner_id                   0
player_of_the_match_id    514
dtype: int64

In [ ]:
train_match = match_df[match_df['match id'].isin(train['match id'])]

In [ ]:
train_match['match id'].nunique()

948

In [ ]:
train_match.loc[741, 'team1_roster_ids']

'9373356.0:7857520.0:4232164.0:4566540.0:3299407.0:232000.0:7883504.0:8725723.0:7878989.0:2526390.0:8725730.0:4017523.0'

In [ ]:
train_match.venue.nunique()

125

In [ ]:
train_match.isna().sum()

match id                    0
team1                       0
team2                       0
winner                      0
by                          0
win amount                  0
toss winner                 0
toss decision               0
venue                       0
city                        0
match_dt                    0
lighting                    0
series_name                 0
season                      0
ground_id                   0
umpire1                     1
umpire2                     1
inning1_runs                0
inning1_wickets             0
inning1_balls               0
inning2_runs                0
inning2_wickets             0
inning2_balls               0
team1_id                    0
team1_roster_ids            0
team2_id                    0
team2_roster_ids            0
series_type                 0
winner_id                   0
player_of_the_match_id    282
dtype: int64

## Training data

In [ ]:
train['lighting'].unique()

array(['day/night match', 'day match', 'night match'], dtype=object)

In [ ]:
train.team1_id.unique()

array([11283,    20, 10576, 36084, 30407, 42573,    27, 47501, 12634,
         216, 30435, 30400,  8301, 10618, 30393, 33956,   293,  7258,
        1350, 40424, 18360, 47508,  8917, 10366, 17583, 47480, 40452,
       12389,    55, 44904, 33963, 46773,    48,    62, 48929,   202,
       17653,  9876,  8987, 45072, 48922,  6698, 40578,   895, 45933,
       17982,   188, 23869, 47529, 23750,  9967,   251, 48334,  8056,
          69, 46731,    76, 22497,    41, 30414,  9701, 48341, 12046,
       33928,  6838, 11591,  8700, 40606, 48950, 48936, 13971, 48733,
       40298, 30428, 15623, 13474, 33921, 46745, 33942, 36014,    90,
       33935, 36112, 45954, 14629, 17744,  7608,  7727, 18570,   825,
       46738, 36126, 30421,   769, 46759,   209,   174, 46766, 33949,
       33914, 23113, 46780,  8182,  1224,   195, 11157, 14454, 36098,
       49657, 15301, 47494, 36070,    34, 45940, 23316, 40592, 23841,
        7573,   958, 40564, 22763, 12473, 40550, 13131, 22784, 47487,
         146, 38814,

In [ ]:
train.columns

Index(['match id', 'team1', 'team1_id', 'team1_roster_ids', 'team2',
       'team2_id', 'team2_roster_ids', 'winner', 'winner_id', 'toss winner',
       'toss decision', 'venue', 'city', 'match_dt', 'lighting', 'series_name',
       'season', 'ground_id', 'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15'],
      dtype='object')

In [ ]:
train.ground_id.nunique()

123

In [ ]:
train.isna().sum()

match id                      0
team1                         0
team1_id                      0
team1_roster_ids              0
team2                         0
team2_id                      0
team2_roster_ids              0
winner                        0
winner_id                     0
toss winner                   0
toss decision                 0
venue                         0
city                          0
match_dt                      0
lighting                      0
series_name                   0
season                        0
ground_id                     0
team_count_50runs_last15      0
team_winp_last5               0
team1only_avg_runs_last15    21
team1_winp_team2_last15       0
ground_avg_runs_last15       53
dtype: int64

In [ ]:
train['winner_01'] = train.apply(lambda x: 1 if (x['team1']==x['winner']) else 0, axis=1)

In [ ]:
train['toss_winner_01'] = np.where(train['toss winner']==train['team1'], 1, 0)
test['toss_winner_01'] = np.where(test['toss winner']==test['team1'], 1, 0)

In [ ]:
train['toss_decision_01'] = np.where(train['toss decision']=='bat', 1, 0)
test['toss_decision_01'] = np.where(test['toss decision']=='bat', 1, 0)

## Test data

In [ ]:
test.columns

Index(['match id', 'team1', 'team1_id', 'team1_roster_ids', 'team2',
       'team2_id', 'team2_roster_ids', 'toss winner', 'toss decision', 'venue',
       'city', 'match_dt', 'lighting', 'series_name', 'season', 'ground_id',
       'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15'],
      dtype='object')

In [ ]:
test

948

In [ ]:
len(test)

271

In [ ]:
test['match id'].nunique()

948

# Feature engineering

## Utility functions

In [ ]:
def giveLastNgamesPlayer(player_id, date, n, bat_or_bowl):
    '''
    Function to get last n games stats of a player before an input date.

    Input-
    1. player_id: id of the player to get historical data.
    2. date: date to look-back and get n games. Stats returned are before this input date.
    3. n: Number of historical games stats to return.
    4. bat_or_bowl: Kind of stats to return. {'bat': batting stats to return, 'bowl': bowling stats to return}

    Output-None

    Returns- dataframe having bowling/batting stats from last n games of a player before an input date.
    The results are sorted by date.
    '''
    if bat_or_bowl == 'bat':
        df_topick = bt_df
        id_col = 'batsman_id'
    else:
        df_topick = bwl_df
        id_col = 'bowler_id'

    return df_topick[(df_topick['match_dt']<date) & (df_topick[id_col]==float(player_id))].sort_values(by='match_dt', ascending=False).head(n)


def giveAllPrevgamesPlayer(player_id, date, bat_or_bowl):
    '''
    Function to get all previous games stats of a player before an input date.

    Input-
    1. player_id: id of the player to get historical data.
    2. date: date to look-back and get n games. Stats returned are before this input date.
    3. n: Number of historical games stats to return.
    4. bat_or_bowl: Kind of stats to return. {'bat': batting stats to return, 'bowl': bowling stats to return}

    Output-None

    Returns- dataframe having bowling/batting stats from all previous games of a player before an input date.
    The results are sorted by date.
    '''
    if bat_or_bowl == 'bat':
        df_topick = bt_df
        id_col = 'batsman_id'
    else:
        df_topick = bwl_df
        id_col = 'bowler_id'

    return df_topick[(df_topick['match_dt']<date) & (df_topick[id_col]==float(player_id))].sort_values(by='match_dt', ascending=False)

#feature 2
def no50_100Lastn(player_list, date, n):

    player_list = str(player_list).split(':') # split string of ':' separated ids into a list of ids
    res_list = []
    for player in player_list: # loop over each player_id in roster
        df_rel = giveLastNgamesPlayer(player_id=player, date=date, n=n, bat_or_bowl='bat') # getting batting stats from last n games for each player.
        df_rel['gte_50runs'] = np.where(df_rel['runs']>=50, 1, 0) # binary indicator to denote whether the player scored atleast 50 in the game (runs>=50).
        res_list.append(np.nansum(df_rel['gte_50runs']))# Sum up number of 50s and 100s for the player and append to a list. We will do this for all players.
    return np.nansum(res_list)# Sum up values of the list which is sum of 50s and 100s by all players in the roster.

In [ ]:
# feature 3
def diff_in_wktkrs(team1_rost, team2_rost, date):
    team1_list = str(team1_rost).split(':')
    team2_list = str(team2_rost).split(':')
    wktkr_1, wktkr_2 = 0, 0
    for player in team1_list:
        wktkrs = giveAllPrevgamesPlayer(player, date, bat_or_bowl='bat')['bowler_id'].dropna().unique()
        n_playing = [1 if str(x) in team2_rost else 0 for x in wktkrs]
        wktkr_1 += np.sum(n_playing)
    for player in team2_list:
        wktkrs = giveAllPrevgamesPlayer(player, date, bat_or_bowl='bat')['bowler_id'].dropna().unique()
        n_playing = [1 if str(x) in team2_rost else 0 for x in wktkrs]
        wktkr_2 += np.sum(n_playing)
    outp = wktkr_1 - wktkr_2
    return outp

# feature 4 - returns about 30% NaNs
def diff_avg_score_at_ground(team1id, team2id, ground_id, date):
    team1_df = match_df[(match_df['match_dt']<date)&                                              # time stipulation
                        ((match_df['team1_id']==team1id)|(match_df['team2_id']==team1id))&        # selecting for team1
                        (match_df['ground_id']==ground_id)]                                       # all previous games at given ground

    team2_df = match_df[(match_df['match_dt']<date)&                                              # time stipulation
                        ((match_df['team1_id']==team2id)|(match_df['team2_id']==team2id))&        # selecting for team2
                        (match_df['ground_id']==ground_id)]                                       # all previous games at given ground


    runs_team1_tossw = team1_df[(team1_df['tosswin_team']==1)&(team1_df['toss decision']=='bat')]['inning1_runs'].sum()
    runs_team1_tossl = team1_df[(team1_df['tosswin_team']==0)&(team1_df['toss decision']=='field')]['inning1_runs'].sum()

    runs_team2_tossw = team2_df[(team2_df['tosswin_team']==1)&(team2_df['toss decision']=='bat')]['inning1_runs'].sum()
    runs_team2_tossl = team2_df[(team2_df['tosswin_team']==0)&(team2_df['toss decision']=='field')]['inning1_runs'].sum()

    outp = sum([runs_team1_tossw,runs_team1_tossl]) - sum([runs_team2_tossw, runs_team2_tossl])

    if (sum([runs_team1_tossw,runs_team1_tossl])>0) & (sum([runs_team2_tossw, runs_team2_tossl])>0):
      return outp
    else:
      return np.nan       # fix later

# feature 5 - returning NaNs
def team1_optimal_toss(tosswinteam, tossdec, date, ground_id):
    df_ground = match_df[(match_df['match_dt']<date)&                                           # time stipulation
                        (match_df['ground_id']==ground_id)]                                     # all previous games at given ground

    bt_first_wins = len(match_df[(match_df['tosswin_team']==1)&(match_df['winner_id']==match_df['team1_id'])&(match_df['toss decision']=='bat')]) + len(match_df[(match_df['tosswin_team']==0)&(match_df['winner_id']==match_df['team2_id'])&(match_df['toss decision']=='bat')])
    bwl_first_wins = len(match_df[(match_df['tosswin_team']==1)&(match_df['winner_id']==match_df['team1_id'])&(match_df['toss decision']=='field')]) + len(match_df[(match_df['tosswin_team']==0)&(match_df['winner_id']==match_df['team2_id'])&(match_df['toss decision']=='field')])

    if bt_first_wins>bwl_first_wins:
        optimal = 1
    else:
        optimal = 0

    if optimal==1:
      if tosswinteam==1 & tossdec=='bat':
        return 1
      elif tosswinteam==0 & tossdec=='bat':
        return 0
    else:
      if (tosswinteam==1) & (tossdec=='field'):
        return 1
      elif (tosswinteam==0) & (tossdec=='field'):
        return 0

In [ ]:
# feature 6 - reconsider
def tosswinner_matchwinner(team1id, team2id, tosswinteam, date):
    team1_df = match_df[(match_df['match_dt']<date)&                                              # time stipulation
                        ((match_df['team1_id']==team1id)|(match_df['team2_id']==team1id))]        # selecting for team1
    team2_df = match_df[(match_df['match_dt']<date)&                                              # time stipulation
                        ((match_df['team1_id']==team2id)|(match_df['team2_id']==team2id))]        # selecting for team2

    team1_wins_tossw = len(team1_df[(((team1_df['tosswin_team']==1)&(team1_df['team1_id']==team1id))|((team1_df['tosswin_team']==0)&(team1_df['team2_id']==team1id)))&(team1_df['winner_id']==team1id)])
    team2_wins_tossw = len(team2_df[(((team2_df['tosswin_team']==1)&(team2_df['team1_id']==team2id))|((team2_df['tosswin_team']==0)&(team2_df['team2_id']==team2id)))&(team2_df['winner_id']==team2id)])

    if tosswinteam==1:
      if team1_wins_tossw>team2_wins_tossw:
          return 1
      else:
          return 0
    else:
      if team1_wins_tossw<team2_wins_tossw:
          return 0
      else:
          return 1

# feature 7 - rediscuss


# feature 9

def diff_strk1_econ2(team1rost, team2rost, date, n):

    team1_list = str(team1rost).split(':')
    team2_list = str(team2rost).split(':')

    strk1, econ2 = [], []
    for player in team1_list: # loop over each player_id in roster
        strk_rt_avg = giveLastNgamesPlayer(player, date, n, 'bat')['strike_rate'].mean()
        if not np.isnan(strk_rt_avg): strk1.append(strk_rt_avg)

    for player in team2_list: # loop over each player_id in roster
        econ_rt_avg = giveLastNgamesPlayer(player, date, n, 'bowl')['economy'].dropna().mean()
        if not np.isnan(econ_rt_avg): econ2.append(econ_rt_avg)

    outp = (np.mean(strk1) + np.mean(econ2))/2
    return outp

# feature 10

def diff_avg_catches(team1rost, team2rost, date, n):
    pass


In [ ]:
# feature 11
def diff_bunnies(team1rost, team2rost, date, n):
    team1_list = str(team1rost).split(':')
    team2_list = str(team2rost).split(':')

    bun_bwl1, bun_bwl2 = [], []
    for player in team1_list: # loop over each player_id in roster
        top2_bwl = list(giveLastNgamesPlayer(player, date, n, 'bat')['bowler_id'].value_counts().index[:2])
        bun_bwl2.extend(top2_bwl)

    for player in team2_list: # loop over each player_id in roster
        top2_bwl = list(giveLastNgamesPlayer(player, date, n, 'bat')['bowler_id'].value_counts().index[:2])
        bun_bwl1.extend(top2_bwl)

    n_bunbwl1 = len([x for x in bun_bwl1 if x in team1_list])
    n_bunbwl2 = len([x for x in bun_bwl2 if x in team2_list])
    outp = n_bunbwl1 - n_bunbwl2
    return outp

# feature 13
def diff_potm_winners(team1rost, team2rost, date):
    team1_list = str(team1rost).split(':')
    team2_list = str(team2rost).split(':')

    match_bef = match_df[(match_df['match_dt']<date)]

    n_team1_potms = len(match_bef[match_bef['player_of_the_match_id'].isin(team1_list)])      # using number of PoTM awards won by players in the squad, rather than
    n_team2_potms = len(match_bef[match_bef['player_of_the_match_id'].isin(team2_list)])      # number of players who have won (at least one) PoTM award

    outp = n_team1_potms - n_team2_potms
    return outp


# feature 15
def weighted_bowlstyle_weakness(team1_rost, team2_rost, date):
    team1_list = str(team1_rost).split(':')
    team2_list = str(team2_rost).split(':')
    wktkr_1, wktkr_2 = 0, 0
    weakstyles_1, weakstyles_2 = [], []

    playing_styles_1 = [bwl_df[bwl_df['bowler_id']==player].loc[bwl_df[bwl_df['bowler_id']==player].index[0], 'bowler_details'].split(":")[-2] if len(bwl_df[bwl_df['bowler_id']==player])>0 else 'btsm' for player in team1_list]
    playing_styles_2 = [bwl_df[bwl_df['bowler_id']==player].loc[bwl_df[bwl_df['bowler_id']==player].index[0], 'bowler_details'].split(":")[-2] if len(bwl_df[bwl_df['bowler_id']==player])>0 else 'btsm' for player in team2_list]

    for player in team1_list:
        styles = list(giveAllPrevgamesPlayer(player, date, bat_or_bowl='bat')['bowler_details'].str.split(":").str[-2].value_counts()[:1])
        weakstyles_1.extend(styles)

    for player in team2_list:
        styles = list(giveAllPrevgamesPlayer(player, date, bat_or_bowl='bat')['bowler_details'].str.split(":").str[-2].value_counts()[:1])
        weakstyles_2.extend(playing_styles_2)

    weakbts_1 = len([x for x in weakstyles_1 if x in playing_styles_2])
    weakbts_2 = len([x for x in weakstyles_2 if x in playing_styles_1])

    outp = (weakbts_1 + weakbts_2) / 2
    return outp

    # complete

May be incorrect

In [ ]:
#feature 14
def lighting_advg(team1id, team2id, ground_id, date):
    team1_df = match_df[(match_df['match_dt']<date)&                                              # time stipulation
                        ((match_df['team1_id']==team1id)|(match_df['team2_id']==team1id))&        # selecting for team1
                        (match_df['ground_id']==ground_id)]                                       # all previous games at given ground

    team2_df = match_df[(match_df['match_dt']<date)&                                              # time stipulation
                        ((match_df['team1_id']==team2id)|(match_df['team2_id']==team2id))&        # selecting for team2
                        (match_df['ground_id']==ground_id)]                                       # all previous games at given ground

    lighting_team1_n = team1_df[(team1_df['winner_id']==team1id)&(team1_df['lighting']=='night match')].sum()
    lighting_team1_d= team1_df[(team1_df['winner_id']==team1id)&(team1_df['lighting']=='day match')].sum()
    lighting_team1_dn = team1_df[(team1_df['winner_id']==team1id)&(team1_df['lighting']=='day/night match')].sum()

    lighting_team2_n = team2_df[(team2_df['winner_id']==team2id)&(team1_df['lighting']=='night match')].sum()
    lighting_team2_d= team2_df[(team2_df['winner_id']==team2id)&(team1_df['lighting']=='day match')].sum()
    lighting_team2_dn = team2_df[(team2_df['winner_id']==team2id)&(team1_df['lighting']=='day/night match')].sum()

    if lighting_team1_n > lighting_team1_d and lighting_team1_n > lighting_team1_dn:
        return "Greater prob for winning at night matches"
    elif lighting_team1_d > lighting_team1_n and lighting_team1_d > lighting_team1_dn:
        return "Greater prob for winning at day matches"
    elif lighting_team1_dn > lighting_team1_n and lighting_team1_dn > lighting_team1_d:
        return "Greater prob for winning at day/night matches"
    else:
        return "Greater prob for winning at day/night matches"


    if lighting_team2_n > lighting_team2_d and lighting_team2_n > lighting_team2_dn:
        return "Greater prob for winning at night matches"
    elif lighting_team2_d > lighting_team2_n and lighting_team2_d > lighting_team2_dn:
        return "Greater prob for winning at day matches"
    elif lighting_team2_dn > lighting_team2_n and lighting_team2_dn > lighting_team2_d:
        return "Greater prob for winning at day/night matches"
    else:
        return "No clear lighting advantage"

In [ ]:
# feature 15
def best_team_season():

      # Group by season and winner, then count the occurrences
        count_winner = match_df.groupby(['season', 'winner_id']).size().reset_index(name='counts')
      # Find the winner with the maximum occurrences for each season
        max_winners = count_winner.loc[count_winner.groupby('season')['counts'].idxmax()]
        return max_winners(winner_id)

## Adding features

Helper features (drop at the end):

In [ ]:
match_df['tosswin_team'] = match_df.apply(lambda x: 1 if (x['toss winner']==x['team1']) else 0, axis=1)
train['tosswin_team'] = train.apply(lambda x: 1 if (x['toss winner']==x['team1']) else 0, axis=1)
# test['tosswin_team'] = test.apply(lambda x: 1 if (x['toss winner']==x['team1']) else 0, axis=1)
train['winner_01'] = train.apply(lambda x: 0 if (x['team1']==x['winner']) else 1, axis=1)

Train features:

In [ ]:
train['team1_count_50_100s_last15'] = train.progress_apply(lambda x: no50_100Lastn(player_list=x['team1_roster_ids'], date=x['match_dt'], n=15), axis=1)

100%|██████████| 948/948 [00:43<00:00, 21.82it/s]


In [ ]:
train['team2_count_50_100s_last15'] = train.progress_apply(lambda x: no50_100Lastn(player_list=x['team2_roster_ids'], date=x['match_dt'], n=15), axis=1)

100%|██████████| 948/948 [00:44<00:00, 21.41it/s]


In [ ]:
train['diff_50or100_last15'] = (train['team1_count_50_100s_last15']+1)/(train['team2_count_50_100s_last15']+1)
train.drop(columns=['team1_count_50_100s_last15','team2_count_50_100s_last15'], inplace=True)


In [ ]:
train['diff_potm_winners'] = train.progress_apply(lambda x: diff_potm_winners(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

100%|██████████| 948/948 [00:02<00:00, 423.91it/s]


In [ ]:
train['diff_in_wktkrs'] = train.progress_apply(lambda x: diff_in_wktkrs(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

100%|██████████| 948/948 [01:34<00:00, 10.06it/s]


In [ ]:
train['diff_avg_score_ground'] = train.progress_apply(lambda x: diff_avg_score_at_ground(x['team1_id'], x['team2_id'], x['ground_id'], x['match_dt']), axis=1)

100%|██████████| 948/948 [00:08<00:00, 115.54it/s]


In [ ]:
train['team1_optimal_toss'] = train.progress_apply(lambda x: team1_optimal_toss(x['tosswin_team'], x['toss decision'], x['match_dt'], x['ground_id']), axis=1)

100%|██████████| 948/948 [00:06<00:00, 143.88it/s]


In [ ]:
train['tosswin_matchwin'] = train.progress_apply(lambda x: tosswinner_matchwinner(x['team1_id'], x['team2_id'], x['tosswin_team'], x['match_dt']), axis=1)

100%|██████████| 948/948 [00:05<00:00, 180.78it/s]


In [ ]:
train['diff_strk1_econ2'] = train.progress_apply(lambda x: diff_strk1_econ2(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt'], n=15), axis=1)

  2%|▏         | 20/948 [00:01<01:03, 14.66it/s]/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 948/948 [01:12<00:00, 13.01it/s]


In [ ]:
train['diff_bunnies'] = train.progress_apply(lambda x: diff_bunnies(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt'], n=15), axis=1)

100%|██████████| 948/948 [01:37<00:00,  9.74it/s]


In [ ]:
train['bowlstyle_weakness_comp'] = train.progress_apply(lambda x: weighted_bowlstyle_weakness(x['team1_roster_ids'], x['team2_roster_ids'], x['match_dt']), axis=1)

100%|██████████| 948/948 [01:59<00:00,  7.93it/s]


Taking ratio of (number of 50 runs in last 15 games for team1) to (number of 50 runs in last 15 games for team2). Adding 1 to handle divide by zero exceptions.

RnP of team_count_50runs_last15 computed over the train data. Slope denotes ratio of right most bin to left most bin.

In [ ]:
createRnP(train_data, 'team_count_50runs_last15')

In [ ]:
test_data['team1_count_50runs_last15'] = test_data.progress_apply(lambda x: \
            no50sLastn(player_list=x['team1_roster_ids'], date=x['match_dt'], n=15), axis=1)
test_data['team2_count_50runs_last15'] = test_data.progress_apply(lambda x: \
            no50sLastn(player_list=x['team2_roster_ids'], date=x['match_dt'], n=15), axis=1)
test_data['team_count_50runs_last15'] = (test_data['team1_count_50runs_last15'])/(test_data['team2_count_50runs_last15']+1)
test_data.drop(columns=['team1_count_50runs_last15','team2_count_50runs_last15'], inplace=True)
test_data.shape
test_data.head(2)


2. team_winp_last5

## Building area

In [ ]:
bt_df.columns, bwl_df.columns, match_df.columns

(Index(['match id', 'batsman', 'batsman_id', 'batsman_details',
        'is_batsman_captain', 'is_batsman_keeper', 'inning', 'runs',
        'balls_faced', 'over_faced_first', 'wicket kind', 'out_by_bowler',
        'out_by_fielder', 'bowler_id', 'bowler_details', 'is_bowler_keeper',
        'is_bowler_captain', 'strike_rate', 'Fours', 'Sixes', 'match_dt'],
       dtype='object'),
 Index(['match id', 'bowler', 'bowler_id', 'bowler_details',
        'is_bowler_captain', 'is_bowler_keeper', 'inning', 'runs',
        'wicket_count', 'balls_bowled', 'economy', 'maiden', 'dots', 'Fours',
        'Sixes', 'wides', 'noballs', 'match_dt'],
       dtype='object'),
 Index(['match id', 'team1', 'team2', 'winner', 'by', 'win amount',
        'toss winner', 'toss decision', 'venue', 'city', 'match_dt', 'lighting',
        'series_name', 'season', 'ground_id', 'umpire1', 'umpire2',
        'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_runs',
        'inning2_wickets', 'inning2_balls',

In [ ]:
bwl_df['bowler_details'].unique()

array(['NZ:Right-hand bat:Right-arm medium-fast:',
       'NZ:Left-hand bat:Right-arm medium-fast:',
       'NED:Right-hand bat:Right-arm medium-fast:',
       'NZ:Right-hand bat:Legbreak googly:',
       'NZ:Left-hand bat:Slow left-arm orthodox:',
       'NZ:Right-hand bat:Right-arm medium:',
       'NZ:Right-hand bat:Right-arm offbreak:',
       'NZ:Right-hand bat:Right-arm fast-medium:',
       'AUS:Right-hand bat:Right-arm fast-medium:',
       'AUS:Right-hand bat:Right-arm offbreak:',
       'AUS:Right-hand bat:Right-arm medium-fast:',
       'AUS:Right-hand bat:Left-arm fast-medium:',
       'AUS:Right-hand bat:Legbreak:',
       'PAK:Left-hand bat:Slow left-arm orthodox:',
       'AUS:Left-hand bat:Right-arm fast-medium:',
       'AFG:Right-hand bat:Right-arm offbreak:',
       'AUS:Right-hand bat:Legbreak googly:',
       'ENG:Right-hand bat:Right-arm medium-fast:',
       'AUS:Left-hand bat:Slow left-arm orthodox:',
       'ITA:Right-hand bat:Right-arm offbreak:',
       'AUS:

In [ ]:
train.columns

Index(['match id', 'team1', 'team1_id', 'team1_roster_ids', 'team2',
       'team2_id', 'team2_roster_ids', 'winner', 'winner_id', 'toss winner',
       'toss decision', 'venue', 'city', 'match_dt', 'lighting', 'series_name',
       'season', 'ground_id', 'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15', 'tosswin_team', 'diff_50or100_last15'],
      dtype='object')

In [ ]:
bt_df.loc[:, 'bowler_details'].str.split(":").str[-2].value_counts()[:2]

bowler_details
Right-arm fast-medium    902
Right-arm medium-fast    841
Name: count, dtype: int64

In [ ]:
top = []
for playerid in bt_df['batsman_id'].unique():
    top3_bwl = list(bt_df[bt_df['batsman_id']==playerid]['bowler_id'].value_counts().index[:1])
    top.extend(top3_bwl)

In [ ]:
bwl_df[bwl_df['bowler_id']==3083667].loc[bwl_df[bwl_df['bowler_id']==3083667].index[0], 'bowler_details'].split(":")[-2]

'Right-arm medium-fast'

In [ ]:
len(np.unique(top))

380

In [ ]:
bwlct = list(bt_df[bt_df['batsman_id']==player]['bowler_id'].value_counts().index[:3])

In [ ]:
list(bwlct.index[:2])

[2486896.0, 2666705.0]

In [ ]:
player = 7907451
date = '2021-07-02'
wktkrs = giveAllPrevgamesPlayer(player, date, bat_or_bowl='bat')['bowler_id'].dropna().unique()


In [ ]:
n=15
strk_rt_avg = giveLastNgamesPlayer(player, date, n, 'bat')['strike_rate'].mean()



In [ ]:
match_df[(match_df['team1_id']==team1id)&(match_df['ground_id']==13768)]['match_dt']

244    2021-06-09
266    2021-06-13
289    2021-06-17
311    2021-06-22
Name: match_dt, dtype: object

# Models

Splitting data:
- Train dataset (80%)
- Validation dataset (10%)
- Test dataset (10%)

In [ ]:
x_params = ['diff_50or100_last15','team_winp_last5','team1only_avg_runs_last15','team1_winp_team2_last15','ground_avg_runs_last15', 'diff_potm_winners', 'diff_in_wktkrs', 'team1_optimal_toss', 'tosswin_matchwin', 'diff_strk1_econ2', 'diff_bunnies', 'bowlstyle_weakness_comp']

In [ ]:
train.columns

Index(['match id', 'team1', 'team1_id', 'team1_roster_ids', 'team2',
       'team2_id', 'team2_roster_ids', 'winner', 'winner_id', 'toss winner',
       'toss decision', 'venue', 'city', 'match_dt', 'lighting', 'series_name',
       'season', 'ground_id', 'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15', 'diff_in_wktkrs', 'tosswin_team',
       'diff_avg_score_ground', 'team1_optimal_toss', 'tosswin_matchwin',
       'diff_strk1_econ2', 'diff_bunnies', 'bowlstyle_weakness_comp',
       'diff_50or100_last15', 'diff_potm_winners'],
      dtype='object')

In [ ]:
xtrain, xvalid, ytrain, yvalid = TTS(train[x_params], train['winner_01'], test_size=0.2, random_state=10)

We provide `xtrain` and `ytrain` to the model.

Model makes inferences ("weights") as to the relations between `x` and `y`.

Now, we provide `xvalid` to the model and the model outputs predictions (`ypred`).

Then, we check how similar `ypred` is to `yvalid` through numerous indicators.

In [ ]:
xtrain.isna().mean(), xvalid.isna().mean()

(diff_50or100_last15          0.000000
 team_winp_last5              0.000000
 team1only_avg_runs_last15    0.025066
 team1_winp_team2_last15      0.000000
 ground_avg_runs_last15       0.051451
 diff_potm_winners            0.000000
 diff_in_wktkrs               0.000000
 team1_optimal_toss           0.353562
 tosswin_matchwin             0.000000
 diff_strk1_econ2             0.022427
 diff_bunnies                 0.000000
 bowlstyle_weakness_comp      0.000000
 dtype: float64,
 diff_50or100_last15          0.000000
 team_winp_last5              0.000000
 team1only_avg_runs_last15    0.010526
 team1_winp_team2_last15      0.000000
 ground_avg_runs_last15       0.073684
 diff_potm_winners            0.000000
 diff_in_wktkrs               0.000000
 team1_optimal_toss           0.331579
 tosswin_matchwin             0.000000
 diff_strk1_econ2             0.021053
 diff_bunnies                 0.000000
 bowlstyle_weakness_comp      0.000000
 dtype: float64)

In [ ]:
xtrain.fillna(0, inplace=True)
xvalid.fillna(0, inplace=True)

In [ ]:
def train_predict_model(model, xtrain, xvalid, ytrain, yvalid):
  if model=='gbc':
    model = gbc(n_estimators=300, learning_rate=0.1, max_depth=4)
  elif model=='xgbc':
    # model = xgbc(n_estimators=200, max_depth=5, eta=0.1, gamma=20, random_state=69)
    model = xgbc(max_depth=6, eta=0.3, gamma=15)
  elif model=='lgbmc':
    model = lgbmc()
  elif model=='cbc':
    model = cbc(loss_function='Logloss', eval_metric='AUC', iterations=200, random_seed=69, verbose=0)

  model.fit(xtrain, ytrain)
  ypred = model.predict(xvalid)
  return ypred

In [ ]:
def train_predict_showstats(model, xtrain, xvalid, ytrain, yvalid):
  ypred = train_predict_model(model, xtrain, xvalid, ytrain, yvalid)
  print("Accuracy:", acc(yvalid, ypred))
  print(pd.DataFrame(classrep(yvalid, ypred, output_dict=True)).transpose())

In [ ]:
train_predict_showstats('cbc', xtrain, xvalid, ytrain, yvalid)

Learning rate set to 0.040041
0:	total: 15.1ms	remaining: 3.01s
1:	total: 19.4ms	remaining: 1.92s
2:	total: 35.4ms	remaining: 2.32s
3:	total: 48.9ms	remaining: 2.4s
4:	total: 54.7ms	remaining: 2.13s
5:	total: 64.7ms	remaining: 2.09s
6:	total: 71.6ms	remaining: 1.97s
7:	total: 78.2ms	remaining: 1.88s
8:	total: 80.3ms	remaining: 1.7s
9:	total: 82.3ms	remaining: 1.56s
10:	total: 86.3ms	remaining: 1.48s
11:	total: 91.6ms	remaining: 1.44s
12:	total: 93.8ms	remaining: 1.35s
13:	total: 95ms	remaining: 1.26s
14:	total: 96.5ms	remaining: 1.19s
15:	total: 107ms	remaining: 1.24s
16:	total: 114ms	remaining: 1.22s
17:	total: 123ms	remaining: 1.25s
18:	total: 129ms	remaining: 1.23s
19:	total: 141ms	remaining: 1.27s
20:	total: 149ms	remaining: 1.27s
21:	total: 162ms	remaining: 1.31s
22:	total: 165ms	remaining: 1.27s
23:	total: 169ms	remaining: 1.24s
24:	total: 183ms	remaining: 1.28s
25:	total: 187ms	remaining: 1.25s
26:	total: 198ms	remaining: 1.27s
27:	total: 201ms	remaining: 1.23s
28:	total: 203ms	

In [ ]:
train_predict_showstats('gbc', xtrain, xvalid, ytrain, yvalid)

Accuracy: 0.5263157894736842
              precision    recall  f1-score     support
0              0.558442  0.434343  0.488636   99.000000
1              0.504425  0.626374  0.558824   91.000000
accuracy       0.526316  0.526316  0.526316    0.526316
macro avg      0.531433  0.530359  0.523730  190.000000
weighted avg   0.532570  0.526316  0.522252  190.000000


In [ ]:
train_predict_showstats('xgbc', xtrain, xvalid, ytrain, yvalid)

NameError: name 'train_predict_showstats' is not defined

**why no lgbmc stats ?**

In [ ]:
test.columns

Index(['match id', 'team1', 'team1_id', 'team1_roster_ids', 'team2',
       'team2_id', 'team2_roster_ids', 'toss winner', 'toss decision', 'venue',
       'city', 'match_dt', 'lighting', 'series_name', 'season', 'ground_id',
       'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15', 'toss_winner_01', 'toss_decision_01'],
      dtype='object')

In [ ]:
xtest = test[x_params]


KeyError: 'winner_01'

# Aruneema's Workspace

## Reproduce Cheatsheet code

Feature importance

In [ ]:
def createRnP(X_12, feature, N=5, ylim_lb=0.3, ylim_ub=0.7):


In [ ]:
returns- None
      df = X_12.copy()
    df[f'{feature}_bin'] = df[feature].rank(pct=True)//(1/N) # divide feature values for all games in 5 equi-volume buckets.
    df['count'] = 1
    df['team1_win%'] = df['winner_01'].apply(lambda x: 1-x) # invert winner_01 to get team1 winner indicator
    df['team2_win%'] = df['winner_01'].copy()
    df[f'{feature}_min'] = df[feature].copy()
    df[f'{feature}_max'] = df[feature].copy()
    df_g = df.groupby(f'{feature}_bin').agg({'team1_win%':'mean', 'team2_win%':'mean', 'count':'sum', f'{feature}_min':'min',\
                                            f'{feature}_max':'max'}).reset_index()
    N = min(N,df_g.shape[0])
    blue_bar = df_g['team1_win%'].values.tolist()
    ind = np.arange(N)
    # plotting starts
    plt.figure(figsize=(10,5));
    plt.bar(ind, blue_bar, label='Team 1 win%');
    plt.axhline(y=0.5, linewidth=0.5, color='k', linestyle = '--')
    xlabel = re.sub('team_','ratio_',feature)
    plt.xlabel(f'{xlabel} (team1 / team2) bins');
    plt.ylabel('Win %');
    plt.title(f'RnP - {feature} vs win');
    df_g['xticks'] = df_g.apply(lambda x: str(round(x[f'{feature}_min'],2)) + ' - ' + str(round(x[f'{feature}_max'],2)), axis=1)
    plt.xticks(ind, df_g['xticks']);
    plt.ylim([ylim_lb,ylim_ub]);
    plt.legend(loc='best');
    x2,x1 = blue_bar[-1],blue_bar[0]
    slope = x2/x1
    if slope < 1:
        slope = 1/slope
        x1,x2 = x2,x1
    print('slope:', round(x2,2),'/',round(x1,2), '= ',round(slope,2))
    plt.show();


Statistics

In [ ]:
train['toss_winner_01'].value_counts()

toss_winner_01
0    617
1    331
Name: count, dtype: int64

In [ ]:
train['toss_decision_01'].value_counts()

toss_decision_01
0    617
1    331
Name: count, dtype: int64

In [ ]:
train['team_count_50runs_last15'].describe()

count    948.000000
mean       1.202602
std        1.008793
min        0.083333
25%        0.686678
50%        1.000000
75%        1.400000
max       11.000000
Name: team_count_50runs_last15, dtype: float64

## Hyperparameter tuning

In [ ]:
def aruneema_predict_model(model, xtrain, xvalid, ytrain, yvalid):
  model.fit(xtrain, ytrain)
  ypred = model.predict(xvalid)
  print("Accuracy:", acc(yvalid, ypred))
  print(pd.DataFrame(classrep(yvalid, ypred, output_dict=True)).transpose())

In [ ]:
model = gbc(n_estimators=300, learning_rate=0.1, max_depth=4)
aruneema_predict_model(model, xtrain, xvalid, ytrain, yvalid)

Accuracy: 0.5157894736842106
              precision    recall  f1-score     support
0              0.495327  0.582418  0.535354   91.000000
1              0.542169  0.454545  0.494505   99.000000
accuracy       0.515789  0.515789  0.515789    0.515789
macro avg      0.518748  0.518482  0.514930  190.000000
weighted avg   0.519734  0.515789  0.514070  190.000000


In [ ]:
model = xgbc(max_depth=6, eta=0.3, gamma=15)
aruneema_predict_model(model, xtrain, xvalid, ytrain, yvalid)

Accuracy: 0.5789473684210527
              precision    recall  f1-score     support
0              0.547009  0.703297  0.615385   91.000000
1              0.630137  0.464646  0.534884   99.000000
accuracy       0.578947  0.578947  0.578947    0.578947
macro avg      0.588573  0.583972  0.575134  190.000000
weighted avg   0.590323  0.578947  0.573439  190.000000


In [ ]:
model = cbc(loss_function='Logloss', eval_metric='AUC', iterations=200, random_seed=69, verbose=0)
aruneema_predict_model(model, xtrain, xvalid, ytrain, yvalid)

Accuracy: 0.5368421052631579
              precision    recall  f1-score     support
0              0.514019  0.604396  0.555556   91.000000
1              0.566265  0.474747  0.516484   99.000000
accuracy       0.536842  0.536842  0.536842    0.536842
macro avg      0.540142  0.539572  0.536020  190.000000
weighted avg   0.541242  0.536842  0.535197  190.000000


In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [ ]:
model = lgbmc()

In [ ]:
model = cbc(loss_function='Logloss', eval_metric='AUC', iterations=200, random_seed=69, verbose=0)

## Feature Importance

In [ ]:
xgbc(max_depth=6, eta=0.3, gamma=15).fit(xtrain, ytrain)
df_feat_importance = pd.DataFrame({
    'feat_name': xtrain.columns.tolist(),
    'model_feat_imp_train': model.feature_importances_
})
df_feat_importance = df_feat_importance.sort_values(by='model_feat_imp_train', ascending=False)\
                                       .reset_index(drop=True).head(10)
print(df_feat_importance)

                   feat_name  model_feat_imp_train
0   team_count_50runs_last15                   1.0
1             toss_winner_01                   0.0
2           toss_decision_01                   0.0
3            team_winp_last5                   0.0
4  team1only_avg_runs_last15                   0.0
5    team1_winp_team2_last15                   0.0
6     ground_avg_runs_last15                   0.0


In [ ]:
cbc(loss_function='Logloss', eval_metric='AUC', iterations=200, random_seed=69, verbose=0).fit(xtrain, ytrain)
df_feat_importance = pd.DataFrame({
    'feat_name': xtrain.columns.tolist(),
    'model_feat_imp_train': model.feature_importances_
})
df_feat_importance = df_feat_importance.sort_values(by='model_feat_imp_train', ascending=False)\
                                       .reset_index(drop=True).head(10)
print(df_feat_importance)

                   feat_name  model_feat_imp_train
0   team_count_50runs_last15             21.756374
1  team1only_avg_runs_last15             21.176642
2     ground_avg_runs_last15             17.824485
3            team_winp_last5             15.387509
4    team1_winp_team2_last15             13.470517
5             toss_winner_01              5.604934
6           toss_decision_01              4.779538


# Ritwiz

In file 1

In [ ]:
## refactor

df_file1 = pd.concat([test_data[['match id','dataset_type','win_pred_team_id','win_pred_score',] + list(df_feat_importance['feat_name'].head(10))], \
                     train_data[['match id','dataset_type','win_pred_team_id','win_pred_score',] + list(df_feat_importance['feat_name'].head(10))]])

renaming_dict = {}
for i,col in enumerate(list(df_feat_importance['feat_name'].head(10))):
    renaming_dict[col] = f'indep_feat_id{i+1}'
df_file1.rename(columns=renaming_dict, inplace=True)

for i in range(1,11):
    if f'indep_feat_id{i}' not in df_file1.columns:
        df_file1[f'indep_feat_id{i}'] = np.nan

df_file1['train_algorithm'] = algo_name
df_file1['is_ensemble'] = is_ensemble
df_file1['train_hps_trees'] = n_trees
df_file1['train_hps_depth'] = depth
df_file1['train_hps_lr'] = lr

In [ ]:
df_file1.shape
df_file1.head()

In file 2

In [ ]:
feature_desc = {'team_count_50runs_last15':'Ratio of number of 50s by players in team1 to number of 50s by players in team2 in last 15 games',\
               'team_winp_last5':'Ratio of team1\'s win % to team2\'s win % in last 5 games',\
               'ground_avg_runs_last15':'average runs scored in the ground in last 15 games',\
               'team1_winp_team2_last15':'Team1\'s win percentage against Team2 in last 15 games',\
               'team1only_avg_runs_last15':'team1\'s avg inning runs in last 15 games',\

               'season_num' : 'Numerical form of season. Takes 1 for oldest season and increases for latest seasons.',\
               'toss_winner_01' : 'Toss winner to numerical - 1 if team2 wins, else 0',\
               'toss_decision_01' : 'Toss decision - categorical - 1 if winner bats, 0 otherwise'}

In [ ]:
df_feat_importance.rename(index={0:'feat_id'}, inplace=True)
df_file2 = df_feat_importance
df_file2['feat_id'] = [i+1 for i in df_file2.index]
df_file2['feat_rank_train'] = [i+1 for i in df_file2.index]
df_file2 = df_file2.set_index('feat_id')
df_file2['feat_description'] = df_file2['feat_name'].map(feature_desc)

In [ ]:
df_file2


# ## Save

df_file1.to_csv('file1.csv', index=False)
df_file2.to_csv('file2.csv')
